# Preparing the Tiny Shakespeare dataset (character level)

This notebook follows `prepare.py` to generate character-level training data for nanoGPT.

In [ ]:
import os
import pickle
import requests
import numpy as np

### Download the dataset

In [ ]:
input_file_path = os.path.join(os.path.dirname(__file__), 'input.txt')
if not os.path.exists(input_file_path):
    data_url = 'https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt'
    with open(input_file_path, 'w') as f:
        f.write(requests.get(data_url).text)

### Build the character vocabulary

In [ ]:
with open(input_file_path, 'r') as f:
    data = f.read()
chars = sorted(list(set(data)))
vocab_size = len(chars)
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
def encode(s):
    return [stoi[c] for c in s]
def decode(l):
    return ''.join([itos[i] for i in l])

### Create splits and encode

In [ ]:
n = len(data)
train_data = data[:int(n*0.9)]
val_data = data[int(n*0.9):]
train_ids = encode(train_data)
val_ids = encode(val_data)
print(f'train has {len(train_ids):,} tokens')
print(f'val has {len(val_ids):,} tokens')

### Write binary files and save metadata

In [ ]:
train_ids = np.array(train_ids, dtype=np.uint16)
val_ids = np.array(val_ids, dtype=np.uint16)
train_ids.tofile(os.path.join(os.path.dirname(__file__), 'train.bin'))
val_ids.tofile(os.path.join(os.path.dirname(__file__), 'val.bin'))
meta = {
    'vocab_size': vocab_size,
    'itos': itos,
    'stoi': stoi,
}
with open(os.path.join(os.path.dirname(__file__), 'meta.pkl'), 'wb') as f:
    pickle.dump(meta, f)

This process yields `train.bin`, `val.bin` and a `meta.pkl` file containing the vocabulary.